# Vanilla CoxCC

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pycox
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn_pandas import DataFrameMapper
import torch
import torchtuples as tt
from pycox.datasets import metabric
from pycox.evaluation import EvalSurv
from pycox.models import CoxCC
import warnings
warnings.filterwarnings('ignore')
import joblib
from pycox.simulations import SimStudyNonLinearNonPH
from generate_data import load_data
from scipy.spatial.distance import pdist, squareform
from weighted_conformal_prediction_coxcc import WeightedConformalPrediction
from datetime import datetime
from pycox.datasets import metabric,support

In [3]:
import joblib

class WeightedConformalPrediction():
    def __init__(self,df_train,train_frac = 0.8,num_nodes=[32,32],
                 out_features=1,batch_norm=True,
                 batch_size=128,dropout=0.1,output_bias=False,
                epochs = 512, callbacks = [tt.callbacks.EarlyStopping()],
                 verbose = True,classification_model='LR',
                 percentile = 0.95,epsilon=0.01):
        self.df_train = df_train
        # self.p_t = len(self.df[self.df['event']==1])/len(self.df)
        self.train_frac = train_frac
        # self.cali_frac = calibration_frac
        self.num_nodes = num_nodes
        self.out_features = out_features
        self.batch_norm = batch_norm
        self.batch_size = batch_size
        self.dropout = dropout
        self.output_bias = output_bias
        self.epochs = epochs
        self.callbacks = callbacks
        self.verbose = verbose
        self.clf_model = classification_model
        self.epsilon = epsilon
        self.percentile = percentile
        self.V = None
        self.W = None
        self.p_hat = None
        self.T_h = None
        self.x_mapper = None
        self.get_target = lambda df: (df['duration'].values, df['event'].values)
        self.bh = None
        self.model = None
        
    # 划分数据，输入原始数据，选择划分的比例，输出训练集验证集和calibration set
    def split_data(self):
        random_idx = np.random.permutation(range(len(self.df_train)))
        train_idx = random_idx[:int(len(self.df_train)*self.train_frac)]
        val_idx = random_idx[int(len(self.df_train)*self.train_frac):]
        self.Z_tr = self.df_train.iloc[train_idx,:]
        self.Z_val = self.df_train.iloc[val_idx,:]
    
    def standardize(self):
        cols_standardize = ['x0', 'x7', 'x8','x9','x10','x11','x12','x13']
        cols_leave = ['x1', 'x2', 'x3', 'x4','x5','x6']
        # cols_standardize = ['x0', 'x1', 'x2', 'x3', 'x8']
        # cols_leave = ['x4', 'x5', 'x6', 'x7']
#         cols_standardize = ['x0','x1','x2']
        standardize = [([col], StandardScaler()) for col in cols_standardize]
        leave = [(col, None) for col in cols_leave]
        # polyfeature = [([col], PolynomialFeatures()) for col in cols_standardize]
        self.x_mapper = DataFrameMapper(standardize+leave)
        
        self.x_train = self.x_mapper.fit_transform(self.Z_tr).astype('float32')
        self.x_val = self.x_mapper.transform(self.Z_val).astype('float32')
        # self.x_ca = self.x_mapper.transform(self.Z_ca).astype('float32')
        
        
        self.y_train = self.get_target(self.Z_tr)
        self.y_val = self.get_target(self.Z_val)
        # self.durations_ca, self.events_ca = self.get_target(self.Z_ca)
        self.val = self.x_val, self.y_val
        self.in_features = self.x_train.shape[1]
        
    def preprocessing(self):
        self.split_data()
        self.standardize()

    def run_preprocessing(self):
        if self.x_mapper == None:
            self.preprocessing()
        
    def neural_network_cox(self):
        self.net = torch.nn.Sequential(
            torch.nn.Linear(self.in_features, 32),
            torch.nn.ReLU(),
            torch.nn.BatchNorm1d(32),
            torch.nn.Dropout(0.1),

            torch.nn.Linear(32, 32),
            torch.nn.ReLU(),
            torch.nn.BatchNorm1d(32),
            torch.nn.Dropout(0.1),

            torch.nn.Linear(32, self.out_features)
        )
        self.model = CoxCC(self.net, torch.optim.Adam)
        self.model.fit(self.x_train, self.y_train, self.batch_size, self.epochs, self.callbacks, self.verbose,
                val_data=self.val, val_batch_size=self.batch_size)
        
    def find_baseline_hazard_non_zero_idx(self):
        if self.model == None:
            self.neural_network_cox()
        self.baseline_hazards = self.model.compute_baseline_hazards()
        self.non_zero_idx = self.baseline_hazards[self.baseline_hazards>0].index[1] # 计算第一个非零元素的索引
        self.bh = self.baseline_hazards.loc[self.non_zero_idx]
        
    def compute_nonconformal_score_single(self,t):
        R = self.Z_tr[self.Z_tr['duration']>=t] # 找到at risk的人的covariates
        if len(R) == 0: # 如果没找到at risk的人就跳过
            return None
        x_R = self.x_mapper.transform(R).astype('float32')
        ch_r = self.model.predict_cumulative_hazards(x_R)
        exp_g_r = ch_r.loc[self.non_zero_idx]/self.bh
        return exp_g_r
    
    # 计算nonconformal score的函数，给定一个预测hazard的模型，training set
    # 和calibration set以及base hazard，输出结果
    def compute_nonconformal_score(self):
        # print('WCP:compute nonconformal score')
        if self.bh == None:
            self.find_baseline_hazard_non_zero_idx()
        Z_ca_1 = self.Z_ca[self.Z_ca['event']==1] # calibration set中发病的样本
        x_ca = self.x_mapper.transform(Z_ca_1).astype('float32')
        durations_test_1, events_test_1 = self.get_target(Z_ca_1)
        cumulative_hazards = self.model.predict_cumulative_hazards(x_ca)
        exp_g = cumulative_hazards.loc[self.non_zero_idx].div(self.bh)
        self.V = list()
        for i in range(len(x_ca)): # nonconformal score
            exp_g_r = self.compute_nonconformal_score_single(durations_test_1[i])
            if exp_g_r is None:
                self.V.append(np.inf)
            else:
                self.V.append(np.log(exp_g[i])-np.log(np.sum(exp_g_r)+exp_g[i]))
        print('[Mean]\t%.2f\t [Std.]\t %.2f\t[Max]\t%.2f\t[Min]\t%.2f'%(np.mean(self.V),np.std(self.V),np.max(self.V),np.min(self.V)))
        self.V = np.array(self.V+[np.inf])
        
    # 计算weight的函数，输入traning set, calibration set以及一个用来估计P(T=1|X=x)的分类模型
    def compute_weight(self):
        # print('WCP:compute weight')
        Z_ca_1 = self.Z_ca[self.Z_ca['event']==1]
        X_tr = self.x_train
        X_ca = self.x_mapper.transform(Z_ca_1).astype('float32')
        C_tr = self.Z_tr['event'] # training set的event,用于之后训练分类模型
        # 根据输入选择分类模型
        if self.clf_model == 'RF':
            from sklearn.ensemble import RandomForestClassifier
            self.clf = RandomForestClassifier(max_depth=6,random_state=0)
        elif self.clf_model == 'LR':
            from sklearn.linear_model import LogisticRegression
            self.clf = LogisticRegression(random_state=0)
        elif self.clf_model == 'XGBoost':
            import xgboost as xgb
            self.clf = xgb.XGBClassifier()
        self.clf.fit(X_tr,C_tr) # 训练分类模型
        p_predict = self.clf.predict_proba(X_ca)[:,1] # 预测p_hat
        p_predict[p_predict<0.1] = 0.1
        p_predict[p_predict>0.9] = 0.9
        print(np.max(p_predict),np.min(p_predict))
        self.W = np.divide(1-p_predict,p_predict) # 估计w_hat
    
    def run_compute_nonconformal_score(self):
        if self.V == None:
            self.compute_nonconformal_score()
        else:
            pass 
        
    def run_conpute_weight(self):
        if self.W == None:
            self.compute_weight()
        else:
            pass

    # 计算normalized weight,输入计算的weight，test point，训练过的分类模型
    def compute_normalized_weight(self,x):
        '''
        x: test point
        '''
        # print('WCP:compute normalized weight')
        p_predict = self.clf.predict_proba(x)[0,1] # 预测test point对应的T=1的概率
        w_predict = self.p_t/p_predict # 估计p_hat
        normalize_term = np.sum(self.W)+w_predict 
        p_hat = self.W/normalize_term # 计算所有病人的p_hat
        p_inf = w_predict/normalize_term # 计算无穷点的weight

        p_hat = np.append(p_hat,[p_inf])
        return p_hat
 
    # 计算对应的置信区间，输入nonconformal score, normalized weight p_hat, p_inf,以及指定的percentile
    def compute_quantile(self,t,p_hat,exp_g_x):
        exp_g_x_r = self.compute_nonconformal_score_single(t)
        if exp_g_x_r is None:
            return 1
        V_x = np.log(exp_g_x)-np.log(np.sum(exp_g_x_r))
        p_hat_leave = p_hat[self.V<=V_x[0]]
        return sum(p_hat_leave)
    
    def weighted_conformal_prediction(self,x,percentile=0.95):
        
        ch = self.model.predict_cumulative_hazards(x)
        exp_g_x = ch.loc[self.non_zero_idx]/self.bh

        p_hat = self.compute_normalized_weight(x)

        if percentile < 0.5:
            quantile = 1
            t = 5
            quantile = self.compute_quantile(t,p_hat,exp_g_x)
            while (quantile > percentile):
                step = t*(quantile-percentile)
                if step < 0.01:
                    step = 0.01
                t = t - sgn*step
                if int(t) < 0:
                    t = 0
                    break
                exp_g_x_r = self.compute_nonconformal_score_single(t)
                V_x = np.log(exp_g_x)-np.log(np.sum(exp_g_x_r))
                quantile = sum(p_hat[self.V<=V_x[0]])
                print(quantile,t)

            t_l = 0
            t_h = t
        else:
            quantile_l,quantile_h = 1,0
            t_l = 5
            t_h = 5
            quantile_l = self.compute_quantile(t_l,p_hat,exp_g_x)
            while (quantile_l>(1-percentile)/2):
                step_l = t_l*(quantile_l-(1-percentile)/2)
                if step_l < 0.01:
                    step_l = 0.01
                t_l = t_l - step_l
                if int(t_l) <= 0:
                    t_l = 0
                    break
                exp_g_x_r = self.compute_nonconformal_score_single(t_l)
                if exp_g_x_r is None:
                    quantile_l = 1
                    continue
                V_x = np.log(exp_g_x)-np.log(np.sum(exp_g_x_r))
                quantile_l = sum(p_hat[self.V<=V_x[0]])
                # print(quantile_l,t_l)
                
            quantile_h = self.compute_quantile(t_h,p_hat,exp_g_x)
            while (quantile_h<(0.5+self.percentile/2)):
                step_h = t_h*(0.5+self.percentile/2-quantile_h)
                if step_h < 0.01:
                    step_h = 0.01
                t_h = t_h + step_h
                exp_g_x_r = self.compute_nonconformal_score_single(t_h)
                if exp_g_x_r is None:
                    quantile_h = 1
                    continue
                V_x = np.log(exp_g_x)-np.log(np.sum(exp_g_x_r))
                # print(V_x)
                quantile_h = sum(p_hat[self.V<=V_x[0]])
            
        return (t_l, t_h)

    def run_training_step(self):
        print('--'*30)
        print('Begin Preprcessing Algorithm 1')
        self.run_preprocessing()
        print('--'*30)
        try:
            self.load_parameters(V_path = './model_data/V_alg_1.txt',W_path='./model_data/W_alg_1.txt',bh_path='./model_data/bh.txt',clf_path='./model_data/clf.model',model_path='./model_data/net_cox.model')
            print('Loading Parameters From Files')
        except:
            print('Begin Noncoformal Score')
            self.run_compute_nonconformal_score()
            print('--'*30)
            print('Begin Compute Wieght')
            self.run_conpute_weight()
            self.save_parmeters(V_path = './model_data/V_alg_1.txt',W_path='./model_data/W_alg_1.txt',bh_path='./model_data/bh.txt',clf_path='./model_data/clf.model',model_path='./model_data/net_cox.model')
        plt.hist(self.W,bins=100)
        plt.savefig('W.pdf')
        print(len(self.W),len(self.V))
            
    def get_T(self,x,percentile=0.95):
        t_l,t_h = self.weighted_conformal_prediction(x,percentile)
        return (t_l,t_h)
    
    def get_nonconformal_score_of_calibration(self):
        if self.V is None:
            self.compute_nonconformal_score()
        return self.V
    
    def get_weight(self):
        if self.W is None:
            self.compute_weight()
        return self.W
    
    def get_normalized_weight(self,x):
        if self.p_hat is None:
            self.compute_normalized_weight(x)
        
        return self.p_hat

    def save_parmeters(self,V_path = 'V_alg_1.txt',W_path='W_alg_1.txt',bh_path='bh.txt',clf_path='clf.model',model_path='net_cox.model'):
        np.savetxt(V_path,self.V)
        np.savetxt(W_path,self.W)
        np.savetxt(bh_path,np.array([self.bh,self.non_zero_idx]))
        joblib.dump(self.clf,clf_path)
        joblib.dump(self.model,model_path)


    def load_parameters(self,V_path = 'V_alg_1.txt',W_path='W_alg_1.txt',bh_path='bh.txt',clf_path='clf.model',model_path='net_cox.model'):
        self.V = np.loadtxt(V_path)
        self.W = np.loadtxt(W_path)
        self.bh = float(np.loadtxt(bh_path)[0])
        self.non_zero_idx = float(np.loadtxt(bh_path)[1])
        self.clf = joblib.load(clf_path)
        self.model = joblib.load(model_path)


In [4]:
alphas = [0.6,0.7,0.8,0.9,0.95]

method = 'vanilla_cox_ph'
data = 'support'

df = support.read_df()
epochs = 10
train_frac = 0.6
test_frac = 0.2
val_frac = 0.2
for alpha in alphas:
    coverage = []
    coverage_censor = []
    coverage_non_censor = []
    interval_len = []
    for epoch in range(epochs):
          rng = np.random.RandomState(epoch)
          shuffle_idx = rng.permutation(range(len(df)))
          train_idx = shuffle_idx[:int(train_frac*len(df))]
          val_idx = shuffle_idx[int(train_frac*len(df)):int((train_frac+val_frac)*len(df))]
          test_idx = shuffle_idx[int((train_frac+val_frac)*len(df)):]

          df_train = df.iloc[train_idx,:]
          df_val = df.iloc[val_idx,:]
          df_test = df.iloc[test_idx,:]

          cols_standardize = ['x0', 'x1', 'x2']

          standardize = [([col], StandardScaler()) for col in cols_standardize]
          polyfeature = [([col], PolynomialFeatures()) for col in cols_standardize]
          x_mapper = DataFrameMapper(standardize+polyfeature)

          x_train = x_mapper.fit_transform(df_train).astype('float32')
          x_val = x_mapper.transform(df_val).astype('float32')
          x_test = x_mapper.transform(df_test).astype('float32')

          get_target = lambda df:(df['duration'].values,df['event'].values)
          y_train = get_target(df_train)
          y_val = get_target(df_val)
          duration_test, event_test = get_target(df_test)

          val = tt.tuplefy(x_val,y_val)

          in_features = x_train.shape[1]
          num_nodes = [32,32]
          out_features = 1
          batch_norm = True
          dropout = 0.1
          output_bias = False

          net = tt.practical.MLPVanilla(in_features=in_features, num_nodes=num_nodes, out_features = out_features, batch_norm = batch_norm, dropout = dropout, output_bias = output_bias)

          model = CoxCC(net,torch.optim.Adam)

          batch_size = 256
          n_epochs = 512
          verbose = False
          callbacks = [tt.callbacks.EarlyStopping()]
          model.fit(x_train,y_train,batch_size,n_epochs,callbacks,verbose,val_data=val.repeat(10).cat())
          _ = model.compute_baseline_hazards()

          surv = model.predict_surv_df(x_test)
          surv_ = (surv<=1-alpha).to_numpy(dtype='int8')
          index = np.array(surv.index)
          multiply_surv = np.transpose(surv_)*index
          multiply_surv_ = np.where(multiply_surv==0,np.max(index),multiply_surv)

          t_predict = multiply_surv_.min(axis = 1)
          diff_predict_true = np.subtract(t_predict,np.array(df_test['duration']))

          cover = sum(diff_predict_true>=0)/len(t_predict)
          censor = 0
          non_censor = 0
          for i in range(len(df_test)):
                if (diff_predict_true[i] >= 0):
                      if (event_test[i]==0):
                            censor += 1
                      else:
                            non_censor += 1

          coverage.append(cover)
          interval_len.append(np.mean(diff_predict_true))
          n_censor = len(df_test) - sum(df_test['event'])
          if n_censor == 0:
                coverage_censor.append(alpha)
                coverage_non_censor.append(cover)
          elif n_censor == len(df_test):
                coverage_non_censor.append(alpha)
                coverage_censor.append(cover)
          else:
                coverage_censor.append(censor/n_censor)
                coverage_non_censor.append(non_censor/(len(df_test)-n_censor))

          print('[%d]\t%.3f\t%.3f\t%.3f'%(epoch,cover,censor,non_censor))
        
    print('Total Coverage Statistics:\t [Mean]%.3f\t[Std.]%.3f\t[Max]%.3f\t[Min]%.3f'%(np.mean(coverage),np.std(coverage),np.max(coverage),np.min(coverage)))
    np.savetxt('./output/vanilla_coxph_interval_'+data+'_'+str(alpha)+'_'+str(epochs)+'.txt',np.array(interval_len))
    np.savetxt('./output/vanilla_coxph_coverage_'+data+'_'+str(alpha)+'_'+str(epochs)+'.txt',np.array(coverage))
    np.savetxt('./output/vanilla_coxph_censor_coverage_'+data+'_'+str(alpha)+'_'+str(epochs)+'.txt',np.array(coverage_censor))
    np.savetxt('./output/vanilla_coxph_non_censor_coverage_'+data+'_'+str(alpha)+'_'+str(epochs)+'.txt',np.array(coverage_non_censor))

[0]	0.700	191.000	1052.000
[1]	0.683	179.000	1033.000
[2]	0.690	173.000	1051.000
[3]	0.678	176.000	1027.000
[4]	0.700	199.000	1044.000
[5]	0.691	183.000	1043.000
[6]	0.697	184.000	1053.000
[7]	0.700	191.000	1051.000
[8]	0.689	171.000	1052.000
[9]	0.673	185.000	1010.000
Total Coverage Statistics:	 [Mean]0.690	[Std.]0.009	[Max]0.700	[Min]0.673
[0]	0.869	395.000	1147.000
[1]	0.852	387.000	1126.000
[2]	0.852	371.000	1141.000
[3]	0.856	371.000	1148.000
[4]	0.867	370.000	1169.000
[5]	0.856	382.000	1137.000
[6]	0.860	386.000	1140.000
[7]	0.872	409.000	1139.000
[8]	0.864	376.000	1157.000
[9]	0.854	389.000	1126.000
Total Coverage Statistics:	 [Mean]0.860	[Std.]0.007	[Max]0.872	[Min]0.852
[0]	0.988	542.000	1212.000
[1]	0.973	542.000	1185.000
[2]	0.963	519.000	1191.000
[3]	0.977	525.000	1210.000
[4]	0.988	526.000	1228.000
[5]	0.974	531.000	1197.000
[6]	0.976	540.000	1193.000
[7]	0.985	559.000	1190.000
[8]	0.975	523.000	1207.000
[9]	0.990	566.000	1191.000
Total Coverage Statistics:	 [Mean]0.979	[S

# CoxCC+WCCI

In [9]:
method = 'cox_cc'
data = 'support'
print('--'*30+'|')
print('[Method]\t'+method+'\t[Dataset]\t'+data+'\t[Alpha]\t'+str(alpha))
df = support.read_df()
max_duration = max(df['duration'])
alphas = [0.95]
for alpha in alphas:
      
      df_data = df
      epochs = 10
      train_frac = 0.8
      empirical_coverage = []
      empirical_coverage_censor = []
      empirical_coverage_non_censor = []
      empirical_coverage_2 = []
      empirical_coverage_censor_2 = []
      empirical_coverage_non_censor_2 = []
      interval_length_1 = []
      interval_length_2 = []
      for epoch in range(epochs):
            rng = np.random.RandomState(epoch)
            shuffle_idx = rng.permutation(range(len(df_data)))
            train_idx = shuffle_idx[:int(train_frac*len(df_data))]
            calib_test_idx = shuffle_idx[int(train_frac*len(df_data)):]
            df_train = df_data.iloc[train_idx,:]
            df_calib_test = df_data.iloc[calib_test_idx,:]
            df_calib_test.reset_index(drop = True,inplace = True)
            print('--'*30+'|')
            print('Initializing Algorithms')
            wcp = WeightedConformalPrediction(df_train,verbose = False,percentile=alpha)
            wcp.run_preprocessing()
            wcp.find_baseline_hazard_non_zero_idx()
            # 对所有可能的时间计算对用的at risk的人，以及相应的exp_g
            print('--'*30+'|')
            print('Pre-compute Hazard At Risk')
            max_duration = np.max(df_data['duration'])
            min_duration = np.min(df_data['duration'])
            step = 0.1
            num_steps = int((max_duration-min_duration)/step)+1

            try:
                  exp_R_list = np.loadtxt('./data/exp_R_'+str(epoch)+'_'+method+'_'+data+'_'+str(alpha)+'.txt')
            except:
                  exp_R_list = []
                  for i in range(num_steps):
                        t = min_duration + i*step
                        at_risk = df_train[df_train['duration']>=t]
                        if len(at_risk) == 0:
                              exp_R_list.append(0)
                        else:
                              x_risk = wcp.x_mapper.transform(at_risk).astype('float32')
                              cumulative_hazards_risk = wcp.model.predict_cumulative_hazards(x_risk)
                              exp_R_list.append(np.sum(cumulative_hazards_risk.loc[wcp.non_zero_idx]/wcp.bh))

                  np.savetxt('./data/exp_R_'+str(epoch)+'_'+method+'_'+data+'_'+str(alpha)+'.txt',np.array(exp_R_list))

            X_test = wcp.x_mapper.transform(df_calib_test)
            duration_test,event_test = wcp.get_target(df_calib_test)

            sq_dists = squareform(pdist(X_test,'sqeuclidean'))
            kernel_weights = np.exp(-sq_dists)

            random_shuffle =rng.permutation(X_test.shape[0])
            n_test_start_idx = X_test.shape[0] // 2
            n_calib_2_start_idx = X_test.shape[0] // 4

            calibration_indices = random_shuffle[:n_test_start_idx]
            calib_2_idx = random_shuffle[n_test_start_idx:n_test_start_idx+n_calib_2_start_idx]
            test_idx = random_shuffle[n_test_start_idx+n_calib_2_start_idx:]
            # 计算 calibration set 的 non conformal score
            print('--'*30+'|')
            print('Begin Computing Nonconformal Score of Alg. 1')
            df_calib = df_calib_test.iloc[calibration_indices,:]
            df_calib_2 = df_calib_test.iloc[calib_2_idx,:]
            df_test = df_calib_test.iloc[test_idx,:]


            df_calib_non_censor = df_calib[df_calib['event']==1]
            duration_test_non_censor,event_test_non_censor = wcp.get_target(df_calib_non_censor)
            calib_non_censor_idx = np.array(df_calib_non_censor.index)
            x_ca_non_censor = wcp.x_mapper.transform(df_calib_non_censor).astype('float32')
            cumulative_hazards_non_censor = wcp.model.predict_cumulative_hazards(x_ca_non_censor)
            exp_g_non_censor = cumulative_hazards_non_censor.loc[wcp.non_zero_idx]/wcp.bh

            try:
                  nonconformal_score_1 = np.loadtxt('./data/nonconformal_alg_1_'+str(epoch)+'_'+method+'_'+data+'_'+str(alpha)+'.txt')
            except:
                  nonconformal_score_1 = []
                  for i in range(len(df_calib_non_censor)):
                        t = int((duration_test_non_censor[i]-min_duration)/step)
                        exp_g_r = exp_R_list[t]
                        nonconformal_score_1.append(np.log(exp_g_non_censor[i])-np.log(exp_g_r))

                  nonconformal_score_1.append(np.inf)
                  nonconformal_score_1 = np.array(nonconformal_score_1)
                  np.savetxt('./data/nonconformal_alg_1_'+str(epoch)+'_'+method+'_'+data+'_'+str(alpha)+'.txt',nonconformal_score_1)

            sort_indices = np.argsort(nonconformal_score_1)
            sorted_nonconformal_score_1 = nonconformal_score_1[sort_indices]
            # 随机选取100个x_0
            print('--'*30+'|')
            print('Begin Testing')
            coverage_CI = []
            censor_coverage = []
            non_censor_coverage = []
            interval_len = []

            coverage_CI_2 = []
            censor_coverage_2 = []
            non_censor_coverage_2 = []
            interval_len_2 = []

            t1 = datetime.now()
            for test_point_idx in test_idx[:100]:

                  weights = kernel_weights[test_point_idx][calib_non_censor_idx]
                  sampling_probs = kernel_weights[test_point_idx][test_idx] # 计算测试集中每个点的抽样概率
                  sampling_probs /= sampling_probs.sum()

                  # 对第二个calibration set中的点计算nonconformalty score
                  try:
                        nonconformal_score_2 = np.loadtxt('./data/nonconformal_alg_2_'+str(test_point_idx)+'_'+str(epoch)+'_'+method+'_'+data+'_'+str(alpha)+'.txt')
                        calibration_duration = np.loadtxt('./data/calibrated_duration_alg_2_'+str(test_point_idx)+'_'+str(epoch)+'_'+method+'_'+data+'_'+str(alpha)+'.txt')
                  except:
                        nonconformal_score_2 = []
                        calibrated_duration = []
                        for calib_point_2 in calib_2_idx:
                              sorted_calib_weights = np.append(weights[sort_indices[:-1]],kernel_weights[calib_point_2,test_point_idx])
                              sorted_calib_weights /= sorted_calib_weights.sum()
                              sorted_calib_weight_cum_sum = np.cumsum(sorted_calib_weights)
                              CI_calib_idx = np.min(np.argwhere(sorted_calib_weight_cum_sum>=alpha))

                              threshold_score_calib = sorted_nonconformal_score_1[int(CI_calib_idx)]

                              x_test = X_test[calib_point_2].reshape(1,-1)
                              cumulative_hazards_calib_2 = wcp.model.predict_cumulative_hazards(x_test)
                              exp_g_calib_2 = cumulative_hazards_calib_2.loc[wcp.non_zero_idx]/wcp.bh
                              exp_g_r_calib_2 = np.exp(np.log(exp_g_calib_2) - threshold_score_calib)
                              candidate_calib_idx = np.argwhere(exp_R_list>=exp_g_r_calib_2[0])

                              if len(candidate_calib_idx) == 0:
                                    t_calib = max_duration
                              else:
                                    t_calib = np.max(candidate_calib_idx)*step + min_duration
                              nonconformal_score_2.append(max(-duration_test[calib_point_2],duration_test[calib_point_2]-t_calib))

                              calibrated_duration.append(t_calib)
                        nonconformal_score_2.append(np.inf)
                        nonconformal_score_2 = np.array(nonconformal_score_2)
                        np.savetxt('./data/nonconformal_alg_2_'+str(test_point_idx)+'_'+str(epoch)+'_'+method+'_'+data+'_'+str(alpha)+'.txt',nonconformal_score_2)

                        np.savetxt('./data/calibrated_duration_alg_2_'+str(test_point_idx)+'_'+str(epoch)+'_'+method+'_'+data+'_'+str(alpha)+'.txt',np.array(calibrated_duration))

                  sort_indices_2 = np.argsort(nonconformal_score_2)
                  sorted_nonconformal_score_2 = nonconformal_score_2[sort_indices_2]

                  # 再选择100个作为test point，以对应的抽样概率抽样
                  included_1 = 0
                  included_censor_1 = 0
                  included_non_censor_1 = 0

                  included_2 = 0
                  included_censor_2 = 0
                  included_non_censor_2 = 0


                  sample_test_idx = rng.choice(test_idx,size=100,p=sampling_probs)
                  num_non_censor = sum(event_test[sample_test_idx])
                  interval_test_1 = []
                  interval_test_2 = []
                  for test_point_idx2 in sample_test_idx:
                        sorted_weights = np.append(weights[sort_indices[:-1]],kernel_weights[test_point_idx2,test_point_idx])
                        sorted_weights /= sorted_weights.sum()
                        sorted_weight_cum_sum = np.cumsum(sorted_weights)
                        CI_idx = np.min(np.argwhere(sorted_weight_cum_sum>=alpha))

                        threshold_score = sorted_nonconformal_score_1[int(CI_idx)]

                        x_test = X_test[test_point_idx2].reshape(1,-1)
                        cumulative_hazards_test = wcp.model.predict_cumulative_hazards(x_test)
                        exp_g_test = cumulative_hazards_test.loc[wcp.non_zero_idx]/wcp.bh
                        exp_g_r_test = np.exp(np.log(exp_g_test) - threshold_score)
                        candidate_idx = np.argwhere(exp_R_list<=exp_g_r_test[0])

                        if len(candidate_idx) == 0:
                              t_test_1 = max_duration
                        else:
                              t_test_1 = np.min(candidate_idx)*step + min_duration
                        interval_test_1.append(t_test_1)
                        if t_test_1 >= duration_test[test_point_idx2]:
                              included_1 += 1
                              if (event_test[test_point_idx2] == 0):
                                    included_censor_1 += 1
                              else:
                                    included_non_censor_1 += 1

                        # 计算测试点在第二个算法下的区间
                        sorted_weights_2 = np.append(weights[sort_indices_2[:-1]],kernel_weights[test_point_idx2,test_point_idx])
                        sorted_weights_2 /= sorted_weights_2.sum()
                        sorted_weight_cum_sum_2 = np.cumsum(sorted_weights_2)
                        CI_idx_2 = np.min(np.argwhere(sorted_weight_cum_sum_2>=alpha))

                        threshold_score_2 = sorted_nonconformal_score_2[int(CI_idx_2)]
                        if threshold_score_2 >= -t_test_1/2:
                              t_test_2 = t_test_1 + threshold_score_2
                        else:
                              t_test_2 = -threshold_score_2
                        if t_test_2 == np.inf:
                            t_test_2 = max_duration
#                         print(t_test_1,t_test_2,threshold_score_2)
                        interval_test_2.append(t_test_2)

                        if t_test_2 >= duration_test[test_point_idx2]:
                              included_2 += 1
                              if (event_test[test_point_idx2] == 0):
                                    included_censor_2 += 1
                              else:
                                    included_non_censor_2 += 1

                  if num_non_censor == 0:
                        non_censor_coverage.append(alpha)
                        non_censor_coverage_2.append(alpha)
                        censor_coverage.append(included_censor_1/(100-num_non_censor))
                        censor_coverage_2.append(included_censor_2/(100-num_non_censor))
                  elif num_non_censor == 100:
                        censor_coverage.append(alpha)
                        censor_coverage_2.append(alpha)
                        non_censor_coverage.append(included_non_censor_1/num_non_censor)
                        non_censor_coverage_2.append(included_non_censor_2/num_non_censor)
                  else:
                        censor_coverage.append(included_censor_1/(100-num_non_censor))
                        censor_coverage_2.append(included_censor_2/(100-num_non_censor))
                        non_censor_coverage.append(included_non_censor_1/num_non_censor)
                        non_censor_coverage_2.append(included_non_censor_2/num_non_censor)

                  coverage_CI.append(included_1/100)
                  interval_len.append(np.mean(interval_test_1))
                  coverage_CI_2.append(included_2/100)
                  interval_test_2 = np.array(interval_test_2)
                  interval_test_2 = interval_test_2[interval_test_2!=np.inf]
                  interval_len_2.append(np.mean(interval_test_2))
            t2 = datetime.now()
            print('Time Elaspsed:\t %.2f s'% ((t2-t1).total_seconds()))
            print('[Epoch%d]\t [Mean]\t %.3f\t[Std.]\t%.3f'%(epoch,np.mean(coverage_CI),np.mean(coverage_CI_2)))
            np.savetxt('./data/coverage_alg_1_'+str(epoch)+'_'+method+'_'+data+'_'+str(alpha)+'.txt',np.array(coverage_CI))
            empirical_coverage.append(np.mean(coverage_CI))
            empirical_coverage_censor.append(np.mean(censor_coverage))
            empirical_coverage_non_censor.append(np.mean(non_censor_coverage))

            np.savetxt('./data/coverage_alg_2_'+str(epoch)+'_'+method+'_'+data+'_'+str(alpha)+'.txt',np.array(coverage_CI_2))
            empirical_coverage_2.append(np.mean(coverage_CI_2))
            empirical_coverage_censor_2.append(np.mean(censor_coverage_2))
            empirical_coverage_non_censor_2.append(np.mean(non_censor_coverage_2))


            interval_length_1.append(np.mean(interval_len))
            interval_length_2.append(np.mean(interval_len_2))

      print('Empirical Coverage of %d Epochs:\t [Alg.1]\t %.3f [Alg.2]\t %.3f'%(epochs,np.mean(empirical_coverage),np.mean(empirical_coverage_2)))
      print('Censor Data of %d Epochs:\t [Alg.1]\t %.3f [Alg.2]\t %.3f'%(epochs,np.mean(empirical_coverage_censor),np.mean(empirical_coverage_censor_2)))
      print('Non-Censor Data of %d Epochs:\t [Alg.1]\t %.3f [Alg.2]\t %.3f'%(epochs,np.mean(empirical_coverage_non_censor),np.mean(empirical_coverage_non_censor_2)))

      np.savetxt('./output/empirical_coverage_alg_1_'+str(epochs)+'_'+method+'_'+data+'_'+str(alpha)+'.txt',empirical_coverage)
      np.savetxt('./output/empirical_coverage_censor_alg_1_'+str(epochs)+'_'+method+'_'+data+'_'+str(alpha)+'.txt',empirical_coverage_censor)
      np.savetxt('./output/empirical_coverage_non_censor_alg_1_'+str(epochs)+'_'+method+'_'+data+'_'+str(alpha)+'.txt',empirical_coverage_non_censor)

      np.savetxt('./output/empirical_coverage_alg_2_'+str(epochs)+'_'+method+'_'+data+'_'+str(alpha)+'.txt',empirical_coverage_2)
      np.savetxt('./output/empirical_coverage_censor_alg_2_'+str(epochs)+'_'+method+'_'+data+'_'+str(alpha)+'.txt',empirical_coverage_censor_2)
      np.savetxt('./output/empirical_coverage_non_censor_alg_2_'+str(epochs)+'_'+method+'_'+data+'_'+str(alpha)+'.txt',empirical_coverage_non_censor_2)

      np.savetxt('./output/interval_length_alg_1_'+str(epochs)+'_'+method+'_'+data+'_'+str(alpha)+'.txt',interval_length_1)
      np.savetxt('./output/interval_length_alg_2_'+str(epochs)+'_'+method+'_'+data+'_'+str(alpha)+'.txt',interval_length_2)

------------------------------------------------------------|
[Method]	cox_cc	[Dataset]	support	[Alpha]	0.9
------------------------------------------------------------|
Initializing Algorithms
------------------------------------------------------------|
Pre-compute Hazard At Risk
------------------------------------------------------------|
Begin Computing Nonconformal Score of Alg. 1
------------------------------------------------------------|
Begin Testing
Time Elaspsed:	 157.97 s
[Epoch0]	 [Mean]	 0.986	[Std.]	0.998
------------------------------------------------------------|
Initializing Algorithms
------------------------------------------------------------|
Pre-compute Hazard At Risk
------------------------------------------------------------|
Begin Computing Nonconformal Score of Alg. 1
------------------------------------------------------------|
Begin Testing
Time Elaspsed:	 188.44 s
[Epoch1]	 [Mean]	 0.983	[Std.]	0.992
----------------------------------------------------